# Рекомендация тарифов

Задача:
На основе данных клиентов оператора сотовой связи проанализировать поведение клиентов и поиск оптимального тарифа

<b>План проекта</b>:
 1. Открыть файл с данными и изучить его.
 2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
 3. Исследовать качество разных моделей, меняя гиперпараметры.
 4. Проверить качество модели на тестовой выборке.
 5. Проверить модели на вменяемость.

## Откроем и изучим файл

In [ ]:
#Импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from joblib import dump
import random

In [ ]:
#Открываем данные
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('C:/Users/Student/Desktop/users_behavior.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<b>Подвывод</b>: Мы загрузили необходимые библиотеки, открыли файл и изучили его. В нашем распоряжении 3214 пользователя и 5 характеристик, которые их описывают. У нас нет пропусков, все типы данных в порядке, названия колонок написаны корректно.

## Разобьём данные на выборки

In [ ]:
#Делим данные на тренировочные и совместные валидационных и тестовых
df_train, df_valid_and_test = train_test_split(df, test_size=0.4, random_state=12345)

In [ ]:
#Делим совместные данные валидационных и тестовых пополам
df_valid, df_test = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345)

In [ ]:
#Делим признаки на целевой и те, по которым будем предсказывать
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
print('Кол-во объектов', features_train.shape[0])

Кол-во объектов 1928


In [ ]:
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']
print('Кол-во объектов', features_valid.shape[0])

Кол-во объектов 643


In [ ]:
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']
print('Кол-во объектов', features_test.shape[0])

Кол-во объектов 643


<b>Подвывод:</b> теперь у нас 3 выборки: обучающая с 60% всех данных, валидационная с 20% и тестовая с 20%.

## Исследуем модели

### Модель решающего дерева

In [ ]:
best_model = None
best_result = 0

for depth in range(1, 6):
    model_des_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучаем модель с заданной глубиной дерева
    model_des_tree.fit(features_train, target_train) # обучаем модель
    predictions_des_tree = model_des_tree.predict(features_valid) # получам предсказания модели
    result_des_tree = accuracy_score(target_valid, predictions_des_tree) # считаем качество модели
    if result_des_tree > best_result:
        best_model = model_des_tree
        best_result = result_des_tree

print("Accuracy лучшей модели:", best_result, 'глубина:', depth)

Accuracy лучшей модели: 0.8009331259720062 глубина: 5


### Модель случайного леса

In [ ]:
best_model = None
best_result = 0

for est in range(1, 51):
    for depth in range(1, 6):
        model_rand_for = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) # обучаем модель с заданным количеством деревьев
        model_rand_for.fit(features_train, target_train) # обучаем модель на тренировочной выборке
        predictions_rand_for = model_rand_for.predict(features_valid) # получам предсказания модели
        result_rand_for = accuracy_score(target_valid, predictions_rand_for) # считаем качество модели
        if result_rand_for > best_result:
            best_model = model_rand_for # сохраним наилучшую модель
            best_result = result_rand_for #  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result, 'Кол-во деревьев:', est, 'Глубина:', depth)

Accuracy наилучшей модели на валидационной выборке: 0.8118195956454122 Кол-во деревьев: 50 Глубина: 5


### Модель логистической регрессии

In [ ]:
model_log_regr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_log_regr.fit(features_train, target_train)
model_log_regr.predict(features_valid)
score = model_log_regr.score(features_valid, target_valid)
dump(model_log_regr, 'model_9_1.joblib')
print("Accuracy на валидационной выборке:", score)

Accuracy на валидационной выборке: 0.7558320373250389


<b>Подвывод:</b> Наилучшие результаты показали две модели со случайными деревьями, 0.8 и 0.81, модель логистической регресиии показала менее точный результат 0.75. Но чтобы быть уверенными, что модель случайных деревьев нам точно подойдёт, мы должны проверить её на тестовой выборке.

## Проверим модель на тестовой выборке

### Решающее дерево

In [ ]:
predictions_des_tree = model_des_tree.predict(features_test) # получам предсказания модели
result_des_tree = accuracy_score(target_test, predictions_des_tree) # считаем качество модели
print("Accuracy на тестовой выборке:", result_des_tree)

Accuracy на тестовой выборке: 0.80248833592535


### Случайный лес

In [ ]:
predictions_rand_for = model_rand_for.predict(features_test) # получам предсказания модели
result_rand_for = accuracy_score(target_test, predictions_rand_for) # считаем качество модели
print("Accuracy на тестовой выборке:", result_rand_for)

Accuracy на тестовой выборке: 0.8055987558320373


### Логистическая регрессия

In [ ]:
model_log_regr.predict(features_test)
score = model_log_regr.score(features_test, target_test)
dump(model_log_regr, 'model_9_1.joblib')
print("Accuracy на тестовой выборке:", score)

Accuracy на тестовой выборке: 0.7620528771384136


<b>Подвывод:</b> Нам удалось добиться точности предсказания выше 0.75 в двух моделях: модель случайного дерева (0.794), модель случайного леса (0.8)

## Проверим модели на адекватность

In [ ]:
rand_list=[]
n=features_valid.shape[0]
for i in range(n):
    rand_list.append(random.randint(0,1))
print(rand_list)

[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 

In [ ]:
print('Точность предсказаний при случайной выборке:', accuracy_score(target_valid, rand_list))

Точность предсказаний при случайной выборке: 0.4774494556765163


<b>Подвывод:</b> Точность всех моделей выше, чем если бы мы использовали для предсказания случайные числа, значит все смодели прошли тест на вменяемость.

## Вывод

Мы загрузили необходимые библиотеки, открыли файл и изучили его, разделили все данные на 3 выборки и построили 3 модели с вариацией гиперпараметров, чтобы найти наиболее точную версию модели. Мы посчитали точность всех моделей, провели итоговую проверку на тестовой выборке и выяснили, что лучшие результаты показали модели со случайными деревьями (модель случайного дерева (0.794), модель случайного леса (0.8)). Также мы провели тест на вменяемость модели, его прошли все модели.